# Example Predictor: Linear Rollout Predictor

This example contains basic functionality for training and evaluating a linear predictor that rolls out predictions day-by-day.

First, a training data set is created from historical case and npi data.

Second, a linear model is trained to predict future cases from prior case data along with prior and future npi data.
The model is an off-the-shelf sklearn Lasso model, that uses a positive weight constraint to enforce the assumption that increased npis has a negative correlation with future cases.

Third, a sample evaluation set is created, and the predictor is applied to this evaluation set to produce prediction results in the correct format.

## Training

In [1]:
import pickle
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

### Copy the data locally

In [2]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local file
DATA_FILE = 'data/OxCGRT_latest.csv'

In [3]:
import os
import urllib.request
if not os.path.exists('data'):
    os.mkdir('data')
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

('data/OxCGRT_latest.csv', <http.client.HTTPMessage at 0x7f0509d150d0>)

In [4]:
# Load historical data from local file
df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [5]:
df.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H6_Flag', 'H7_Vaccination policy', 'H7_Flag',
       'M1_Wildcard', 'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex',
       'StringencyIndexForDisplay', 'StringencyLegacyIndex',
       'StringencyLegacyIndexForDispla

In [6]:
# For testing, restrict training data to that before a hypothetical predictor submission date
# HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-07-31")
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-05-06")
df = df[df.Date <= HYPOTHETICAL_SUBMISSION_DATE]

In [7]:
# Add RegionID column that combines CountryName and RegionName for easier manipulation of data
df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)

In [8]:
# Add new cases column
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

In [9]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
cases_col = ['NewCases']
npi_cols = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
df = df[id_cols + cases_col + npi_cols]

In [10]:
# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))

In [11]:
# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))

In [12]:
df

,CountryName,RegionName,GeoID,Date,NewCases,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
0,Aruba,NaN,Aruba__nan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,NaN,Aruba__nan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,NaN,Aruba__nan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,NaN,Aruba__nan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,NaN,Aruba__nan,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91202,Zimbabwe,NaN,Zimbabwe__nan,2020-05-02,-6.0,3.0,3.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,0.0
91203,Zimbabwe,NaN,Zimbabwe__nan,2020-05-03,0.0,3.0,3.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,0.0
91204,Zimbabwe,NaN,Zimbabwe__nan,2020-05-04,0.0,3.0,3.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
91205,Zimbabwe,NaN,Zimbabwe__nan,2020-05-05,0.0,3.0,3.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0


In [13]:
# Set number of past days to use to make predictions
nb_lookback_days = 21

# Create training data across all countries for predicting one day ahead
X_cols = cases_col + npi_cols
y_col = cases_col
X_samples = []
y_samples = []
geo_ids = df.GeoID.unique()
for g in geo_ids:
    gdf = df[df.GeoID == g]
    all_case_data = np.array(gdf[cases_col])
    all_npi_data = np.array(gdf[npi_cols])

    # Create one sample for each day where we have enough data
    # Each sample consists of cases and npis for previous nb_lookback_days
    nb_total_days = len(gdf)
    for d in range(nb_lookback_days, nb_total_days - 1):
        X_cases = all_case_data[d-nb_lookback_days:d]

        # Take negative of npis to support positive
        # weight constraint in Lasso.
        X_npis = -all_npi_data[d - nb_lookback_days:d]

        # Flatten all input data so it fits Lasso input format.
        X_sample = np.concatenate([X_cases.flatten(),
                                   X_npis.flatten()])
        y_sample = all_case_data[d + 1]
        X_samples.append(X_sample)
        y_samples.append(y_sample)

X_samples = np.array(X_samples)
y_samples = np.array(y_samples).flatten()

In [14]:
# Helpful function to compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))

In [23]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_samples,
                                                    y_samples,
                                                    test_size=0.2)

In [37]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV

xgb1 = xgb.XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X_train, y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 2 folds for each of 9 candidates, totalling 18 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  18 out of  18 | elapsed:  4.6min finished


0.9218636602684334
{'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 5, 'min_child_weight': 4, 'n_estimators': 500, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}


In [44]:
model = xgb.XGBRegressor(colsample_bytree=0.7, learning_rate=0.05, max_depth=5, min_child_weight=4, n_estimators=500, nthread=4, objective='reg:linear', silent=1, subsample=0.7)
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.05, max_delta_step=0, max_depth=5,
             min_child_weight=4, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=4, nthread=4, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, silent=1, subsample=0.7, tree_method=None,
             validate_parameters=False, verbosity=None)

In [45]:
# Evaluate model
train_preds = model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

test_preds = model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

Train MAE: 22.01049804495378
Test MAE: 50.55399752046113


In [46]:
# Save model to file
if not os.path.exists('models'):
    os.mkdir('models')
# with open('models/model.pkl', 'wb') as model_file:
with open('models/model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

## Evaluation

Now that the predictor has been trained and saved, this section contains the functionality for evaluating it on sample evaluation data.

In [37]:
# Reload the module to get the latest changes
import predict
from importlib import reload
reload(predict)
from predict import predict_df

In [38]:
%%time
preds_df = predict_df("2020-08-01", "2020-08-31", path_to_ips_file="../../../validation/data/2020-09-30_historical_ip.csv", verbose=True)


Predicting for Aruba__nan
2020-08-01: 59.36062255525157
2020-08-02: 86.35045739959925
2020-08-03: 106.00011692462029
2020-08-04: 125.98135230643517
2020-08-05: 144.993791329175
2020-08-06: 171.40154448944634
2020-08-07: 196.68858568685908
2020-08-08: 220.2749052519795
2020-08-09: 243.89316468108422
2020-08-10: 267.6505308278968
2020-08-11: 292.3299848131692
2020-08-12: 317.28504370003645
2020-08-13: 341.60679652164356
2020-08-14: 366.35073584081016
2020-08-15: 391.3936733613328
2020-08-16: 416.7074069902301
2020-08-17: 442.25948550633194
2020-08-18: 467.93923746786567
2020-08-19: 493.8326829330999
2020-08-20: 519.9617208396685
2020-08-21: 546.3193144238694
2020-08-22: 572.902303109842
2020-08-23: 599.7001692493895
2020-08-24: 626.7204566068767
2020-08-25: 653.9698696613566
2020-08-26: 681.4498239302334
2020-08-27: 698.2984955094389
2020-08-28: 717.8646612427472
2020-08-29: 739.2639962504306
2020-08-30: 760.5868111230948
2020-08-31: 782.8017137674522

Predicting for Afghanistan__nan
20

2020-08-27: 2590.031905372164
2020-08-28: 2618.723011551787
2020-08-29: 2654.965086371872
2020-08-30: 2690.0238415942035
2020-08-31: 2725.5079915091537

Predicting for Azerbaijan__nan
2020-08-01: 2171.685204309538
2020-08-02: 2774.9685098425857
2020-08-03: 2802.20691467383
2020-08-04: 2867.1517784946886
2020-08-05: 2450.1440280792076
2020-08-06: 2588.2619513160666
2020-08-07: 2764.168080012811
2020-08-08: 2821.453703141138
2020-08-09: 2865.171545108749
2020-08-10: 2863.6623159916935
2020-08-11: 2906.724810547015
2020-08-12: 2971.054286831618
2020-08-13: 3022.7628864328276
2020-08-14: 3069.7281276295594
2020-08-15: 3112.395441859058
2020-08-16: 3158.8851071082627
2020-08-17: 3209.327194435161
2020-08-18: 3259.3332971352143
2020-08-19: 3308.7500773774605
2020-08-20: 3357.9705342096763
2020-08-21: 3407.858223653459
2020-08-22: 3458.6495745219504
2020-08-23: 3509.873434435563
2020-08-24: 3561.385468808839
2020-08-25: 3613.237976931171
2020-08-26: 3665.539061326971
2020-08-27: 3723.25241270


Predicting for Belarus__nan
2020-08-01: 963.8881598558596
2020-08-02: 1289.683501835255
2020-08-03: 1251.37734496691
2020-08-04: 1289.5447847683977
2020-08-05: 1133.407329997236
2020-08-06: 1208.91873722667
2020-08-07: 1303.7893815233183
2020-08-08: 1336.9258179899746
2020-08-09: 1367.182638275186
2020-08-10: 1381.4359776707138
2020-08-11: 1414.5242436896126
2020-08-12: 1456.88780224289
2020-08-13: 1492.5809294865328
2020-08-14: 1526.5031942292235
2020-08-15: 1559.2033558372414
2020-08-16: 1593.7081322631434
2020-08-17: 1630.0085975659827
2020-08-18: 1666.1016322263667
2020-08-19: 1702.0418763564617
2020-08-20: 1738.0823603436397
2020-08-21: 1774.5541617245453
2020-08-22: 1811.5375874872384
2020-08-23: 1848.8181827652043
2020-08-24: 1886.3446442667596
2020-08-25: 1924.1528913502077
2020-08-26: 1962.2883222237338
2020-08-27: 1982.0194736383037
2020-08-28: 2000.2529887864266
2020-08-29: 2027.1185260580446
2020-08-30: 2054.3275145844605
2020-08-31: 2082.0009368680207

Predicting for Beli


Predicting for Central African Republic__nan
2020-08-01: 54.68489353066612
2020-08-02: 80.75724214038458
2020-08-03: 99.5670382304975
2020-08-04: 119.28424161983469
2020-08-05: 139.5474799675958
2020-08-06: 165.77536033807655
2020-08-07: 190.7558819530141
2020-08-08: 214.18805109356754
2020-08-09: 237.71813790473627
2020-08-10: 261.55594684949097
2020-08-11: 286.2084941929698
2020-08-12: 311.07787557004707
2020-08-13: 335.32854306331075
2020-08-14: 360.01384611709483
2020-08-15: 385.0167119653764
2020-08-16: 410.2832968679166
2020-08-17: 435.77817445615625
2020-08-18: 461.40033768915
2020-08-19: 487.2377648348593
2020-08-20: 513.3126605877436
2020-08-21: 539.6154440272185
2020-08-22: 566.1419056001282
2020-08-23: 592.8825091458737
2020-08-24: 619.8453024547725
2020-08-25: 647.0370474930298
2020-08-26: 674.4588869922754
2020-08-27: 691.2488106645221
2020-08-28: 710.7556708606699
2020-08-29: 733.5827508142834
2020-08-30: 757.3680643315024
2020-08-31: 780.9505769932375

Predicting for Ca

2020-08-14: 359.20256818837254
2020-08-15: 384.2016817862968
2020-08-16: 409.4615206969948
2020-08-17: 434.9487367592298
2020-08-18: 460.5633160662245
2020-08-19: 486.3937307934568
2020-08-20: 512.4619026485395
2020-08-21: 538.7576334107339
2020-08-22: 565.2768000836977
2020-08-23: 592.0100449341335
2020-08-24: 618.9654926840662
2020-08-25: 646.14988461485
2020-08-26: 673.5642888422669
2020-08-27: 682.4588967583004
2020-08-28: 689.7597303686357
2020-08-29: 707.0957916680984
2020-08-30: 723.962006068828
2020-08-31: 740.9652369995431

Predicting for Cape Verde__nan
2020-08-01: 71.69294775941854
2020-08-02: 108.25791232167573
2020-08-03: 124.27580858780044
2020-08-04: 149.4885736319107
2020-08-05: 164.0239842187994
2020-08-06: 189.95749710915746
2020-08-07: 216.85471304537472
2020-08-08: 240.5054963544248
2020-08-09: 264.67150511247496
2020-08-10: 288.26571031375653
2020-08-11: 312.90986576742745
2020-08-12: 338.1793297538597
2020-08-13: 362.71307524093726
2020-08-14: 387.66678865552234
2

...

2020-08-03: 2995.9948868657816
2020-08-04: 3261.383830274382
2020-08-05: 2736.784312669845
2020-08-06: 2842.7705771870656
2020-08-07: 3033.323360698539
2020-08-08: 3094.1750887773364
2020-08-09: 3160.112114593653
2020-08-10: 3155.2037474097597
2020-08-11: 3193.4477165277012
2020-08-12: 3260.6466916986624
2020-08-13: 3315.482939579923
2020-08-14: 3367.0001967768344
2020-08-15: 3412.103958351849
2020-08-16: 3460.1357639150233
2020-08-17: 3512.9511180573804
2020-08-18: 3565.620729786913
2020-08-19: 3617.852973001352
2020-08-20: 3669.7213409707233
2020-08-21: 3722.119657731003
2020-08-22: 3775.5065721256165
2020-08-23: 3829.403519059794
2020-08-24: 3883.6305564790664
2020-08-25: 3938.203202238759
2020-08-26: 3993.224775965691
2020-08-27: 4037.90844888837
2020-08-28: 4090.909817713065
2020-08-29: 4147.821569414399
2020-08-30: 4203.620571674074
2020-08-31: 4259.1369860443565

Predicting for United States__Maryland
2020-08-01: 1379.902727667215
2020-08-02: 1807.5226086715936
2020-08-03: 1793.


Predicting for United States__Nebraska
2020-08-01: 750.8879989478212
2020-08-02: 993.5180079004797
2020-08-03: 1000.018677866281
2020-08-04: 1012.1440533221476
2020-08-05: 897.313606527141
2020-08-06: 964.3569263734089
2020-08-07: 1041.5295311508487
2020-08-08: 1075.1900894879489
2020-08-09: 1102.5473823501552
2020-08-10: 1118.3498298495465
2020-08-11: 1149.8116571368228
2020-08-12: 1188.0449525945867
2020-08-13: 1221.297962575822
2020-08-14: 1252.997019243665
2020-08-15: 1283.7754298087914
2020-08-16: 1316.1314984474402
2020-08-17: 1349.908091001125
2020-08-18: 1383.5693703083905
2020-08-19: 1417.1408928351589
2020-08-20: 1450.8297766980857
2020-08-21: 1484.9151118217144
2020-08-22: 1519.4480025296878
2020-08-23: 1554.2594435859921
2020-08-24: 1589.3090699415175
2020-08-25: 1624.625844984298
2020-08-26: 1660.2479923055241
2020-08-27: 1685.3273648153013
2020-08-28: 1713.197374122469
2020-08-29: 1745.9394139275164
2020-08-30: 1777.8049759684727
2020-08-31: 1809.8785990840597

Predictin

2020-08-01: 4549.281357218752
2020-08-02: 6146.654998736234
2020-08-03: 6123.022367000461
2020-08-04: 6022.257668729226
2020-08-05: 5149.2418588981645
2020-08-06: 5440.9854454247115
2020-08-07: 5822.509335360467
2020-08-08: 5921.7306872099225
2020-08-09: 5964.142361867695
2020-08-10: 5932.673068502089
2020-08-11: 6002.121130612603
2020-08-12: 6116.533343004398
2020-08-13: 6201.539988766809
2020-08-14: 6272.190826470396
2020-08-15: 6334.995668514097
2020-08-16: 6407.119717652151
2020-08-17: 6487.557218169922
2020-08-18: 6566.5536488500875
2020-08-19: 6643.574795491363
2020-08-20: 6719.994448094449
2020-08-21: 6797.800475535821
2020-08-22: 6877.34366924033
2020-08-23: 6957.535636099929
2020-08-24: 7038.0381531013245
2020-08-25: 7119.012857227145
2020-08-26: 7200.719653371662
2020-08-27: 7272.389405984876
2020-08-28: 7352.608849396139
2020-08-29: 7438.425623886596
2020-08-30: 7522.616870902911
2020-08-31: 7606.528128428844

Predicting for United States__Rhode Island
2020-08-01: 662.605945

2020-08-01: 106.17351728382157
2020-08-02: 143.5818073129896
2020-08-03: 161.69094138157868
2020-08-04: 183.50462160416708
2020-08-05: 193.62684938925818
2020-08-06: 222.5905462901324
2020-08-07: 250.92954406893438
2020-08-08: 274.99543529829566
2020-08-09: 299.08385768247496
2020-08-10: 322.3712621891723
2020-08-11: 347.4664611252739
2020-08-12: 373.24272451253125
2020-08-13: 398.1135694193972
2020-08-14: 423.32438490448027
2020-08-15: 448.74740083121833
2020-08-16: 474.51226040001023
2020-08-17: 500.5886503708802
2020-08-18: 526.7778686518681
2020-08-19: 553.1662045051851
2020-08-20: 579.7829155759989
2020-08-21: 606.6372588328459
2020-08-22: 633.7306044224026
2020-08-23: 661.0429665948259
2020-08-24: 688.5793141212787
2020-08-25: 716.3474092010015
2020-08-26: 744.3506727979504
2020-08-27: 761.7282183035311
2020-08-28: 781.8279490817722
2020-08-29: 808.2275911204142
2020-08-30: 834.1373201592103
2020-08-31: 859.3810935679144

Predicting for United States__Washington
2020-08-01: 1604.

2020-08-04: 118.325747521373
2020-08-05: 138.73739569980722
2020-08-06: 164.8625203766143
2020-08-07: 189.80741813051853
2020-08-08: 213.18406554910098
2020-08-09: 236.73206533789082
2020-08-10: 260.5898690918989
2020-08-11: 285.22812404305154
2020-08-12: 310.08285481551314
2020-08-13: 334.3195937975345
2020-08-14: 358.9979745206503
2020-08-15: 383.99613405580754
2020-08-16: 409.25426691369194
2020-08-17: 434.73955108568157
2020-08-18: 460.352220844643
2020-08-19: 486.1808679432341
2020-08-20: 512.2473431965996
2020-08-21: 538.541294548505
2020-08-22: 565.05862138168
2020-08-23: 591.7900107319235
2020-08-24: 618.7436061260712
2020-08-25: 645.926143542471
2020-08-26: 673.338672500046
2020-08-27: 690.1191721462201
2020-08-28: 714.9597076511591
2020-08-29: 739.0235285288145
2020-08-30: 762.169426285592
2020-08-31: 786.358925351592

Predicting for Yemen__nan
2020-08-01: 16.68721041970934
2020-08-02: 44.07251853981379
2020-08-03: 27.052564111903642
2020-08-04: 68.48320300532117
2020-08-05: 

In [ ]:
# Check the predictions
preds_df.head()

# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [ ]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip ../../../validation/data/2020-09-30_historical_ip.csv -o predictions/2020-08-01_2020-08-04.csv

In [ ]:
!head predictions/2020-08-01_2020-08-04.csv

# Test cases
We can generate a prediction file. Let's validate a few cases...

In [ ]:
import os
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [ ]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="../../../validation/data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [ ]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="../../../validation/data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [ ]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

In [ ]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

### Check it

In [ ]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")